In [1]:
# TENSORFLOW CHECK
"""
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")
"""

'\n# What version of Python do you have?\nimport sys\n\nimport tensorflow.keras\nimport pandas as pd\nimport sklearn as sk\nimport tensorflow as tf\n\nprint(f"Tensor Flow Version: {tf.__version__}")\nprint(f"Keras Version: {tensorflow.keras.__version__}")\nprint()\nprint(f"Python {sys.version}")\nprint(f"Pandas {pd.__version__}")\nprint(f"Scikit-Learn {sk.__version__}")\ngpu = len(tf.config.list_physical_devices(\'GPU\'))>0\nprint("GPU is", "available" if gpu else "NOT AVAILABLE")\n'

In [2]:
# RUN THE FILES IF THEY HAVEN'T
!python3 minesweeper.py
!python3 DQNsetup.py
# from minesweeper_env import *
import random
import numpy as np
import pandas as pd
import math
import minesweeper
from minesweeper import Board
import DQNsetup
from DQNsetup import *
from collections import deque
from tqdm import tqdm

Init Plugin
Init Graph Optimizer
Init Kernel


Init Plugin
Init Graph Optimizer
Init Kernel


In [3]:
# Environment settings
MEM_SIZE = 50_000 # number of moves to store in replay buffer
MEM_SIZE_MIN = 5 # min number of moves in replay buffer
episodes = 30

# Learning settings
BATCH_SIZE = 4
learn_rate = 0.001
LEARN_DECAY = 0.99975
LEARN_MIN = 0.001
DISCOUNT = 0.1 #gamma

# Exploration settings
epsilon = 0.95
EPSILON_DECAY = 0.99975
EPSILON_MIN = 0.01

# DQN settings
CONV_UNITS = 64 # number of neurons in each conv layer
DENSE_UNITS = 512 # number of neurons in fully connected dense layer
UPDATE_TARGET_EVERY = 5

In [4]:
# INITIALIZING EVERTHING CELL

# Initialize the Board (4x4 matrix)
env = Board(9, 9)

# Set the mines about (2,2)
# Assume user clicked coordinate (2,2) as the first tile
f_row, f_col = np.random.randint(env.rows, size=2)
print("First row: %d, First Col: %d" % (f_row, f_col))
env.set_mines_about(f_row, f_col,10) # set_mines_about(self,row_center,col_center,num_mines)
print("Mines: ")
env.printMines()
print("Board: ")
env.printBoard()
state_im = env.board3D() # board is currently 2D, making it 3D by (row, col, 1)

# Initialize the model 
model = DQN_setup(learn_rate, state_im.shape, env.ntiles, CONV_UNITS, DENSE_UNITS)

# Initialize the model that would always be ahead (looking at the future)
target_model = DQN_setup(learn_rate, state_im.shape, env.ntiles, CONV_UNITS, DENSE_UNITS)
target_model.set_weights(model.get_weights())

replay_memory = deque(maxlen=MEM_SIZE)
target_update_counter = 0

2022-01-09 23:55:26.113539: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-09 23:55:26.113665: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


First row: 6, First Col: 2
Opening ( 6, 2 ) ...
Opening ( 5, 1 ) ...
Progressed ...
Opening ( 5, 2 ) ...
Opening ( 4, 1 ) ...
Progressed ...
Opening ( 4, 2 ) ...
Opening ( 3, 1 ) ...
Progressed ...
Opening ( 3, 2 ) ...
Opening ( 2, 1 ) ...
Opening ( 1, 0 ) ...
Opening ( 0, 0 ) ...
Opening ( 0, 1 ) ...
Opening ( 0, 2 ) ...
Opening ( 0, 3 ) ...
Progressed ...
Opening ( 1, 1 ) ...
Opening ( 1, 2 ) ...
Opening ( 1, 3 ) ...
Progressed ...
Opening ( 2, 2 ) ...
Opening ( 2, 3 ) ...
Progressed ...
Opening ( 3, 3 ) ...
Progressed ...
Progressed ...
Progressed ...
Opening ( 2, 0 ) ...
Opening ( 3, 0 ) ...
Progressed ...
Progressed ...
Progressed ...
Progressed ...
Progressed ...
Progressed ...
Progressed ...
Progressed ...
Opening ( 4, 3 ) ...
Opening ( 3, 4 ) ...
Progressed ...
Opening ( 4, 4 ) ...
Opening ( 3, 5 ) ...
Progressed ...
Opening ( 4, 5 ) ...
Opening ( 3, 6 ) ...
Progressed ...
Opening ( 4, 6 ) ...
Opening ( 3, 7 ) ...
Progressed ...
Opening ( 4, 7 ) ...
Opening ( 3, 8 ) ...
Opening

In [5]:
# PLAY THE GAME!!! (# episodes Games)
for episode in tqdm(range(1,episodes+1), unit='episode'):
    env.reset()
    f_row, f_col = np.random.randint(env.rows, size=2)
    env.set_mines_about(f_row, f_col,15)
    done = False
    ep_reward = 0

    # play until lose
    while not done:
        # get action
        board = env.board3D().reshape(1, env.ntiles)
        print("Board: ", board)

        # Select the unopened tiles
        unopened_tiles = [i for i, value in enumerate(board[0]) if value==-1]
        print("Unopened Tiles: ", unopened_tiles)

        rand = np.random.random() # number from 0 to 1

        if rand < epsilon:
            print("\nUsed Random")
            move = np.random.choice(unopened_tiles)
        else:
            print("\nUsed Model To Predict")
            moves = model.predict(np.reshape(state_im, (1, env.rows, env.cols, 1)))
            print(type(moves))
            print("moves:", moves)
            # Disregard all the opened tiles
            moves[board!=-1] = np.min(moves)
            # Pick a tile with the best probability
            move = np.argmax(moves)


        print("\naction: ", move)
        print("Board: ", env.board)
        print("Mines: ", env.mines)

        # Retrieve the next step and reward
        new_state, reward, done = env.dig(math.floor(move / env.cols), move % env.cols)
        print("\nREWARD: ", reward)
        ep_reward += reward

        # append the data to batch_array
        current_state = env.board3D()
        replay_memory.append((current_state, move, reward, new_state, done))

        # Train
        if len(replay_memory) < MEM_SIZE_MIN:
            print("SKIP")
            continue

        # Get the random sample of batches
        batch = random.sample(replay_memory, BATCH_SIZE)

        # Q table for current model using the batches
        current_states = np.array([transition[0] for transition in batch])
        current_qs_list = model.predict(current_states)
        print(current_qs_list.shape)
        print("\nCurrent Q Table: ", current_qs_list)

        # Q table for future model
        new_current_states = np.array([transition[3] for transition in batch])
        future_qs_list = target_model.predict(new_current_states)
        print(future_qs_list.shape)
        print("\nFuture Q Table: ", future_qs_list)
        
        X_train = [] # Feature
        Y_train = [] # Label

        for i, (current_state, action, reward, new_current_states, done) in enumerate(batch):
            if not done:
                max_future_q = np.max(future_qs_list[i])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            current_qs = current_qs_list[i]
            current_qs[action] = new_q

            X_train.append(current_state)
            Y_train.append(current_qs)

        model.fit(np.array(X_train), np.array(Y_train), batch_size=BATCH_SIZE)

        # updating to determine if we want to update target_model yet
        if done:
            target_update_counter += 1

        if target_update_counter > UPDATE_TARGET_EVERY:
            target_model.set_weights(model.get_weights())
            target_update_counter = 0

        # decay learn_rate
        learn_rate = max(LEARN_MIN, learn_rate*LEARN_DECAY)

        # decay epsilon
        epsilon = max(EPSILON_MIN, epsilon*EPSILON_DECAY)
    print("Episode Reward: ", ep_reward)
print("Number of Wins :", env.n_wins)

  0%|                                                                                                                                       | 0/30 [00:00<?, ?episode/s]

Opening ( 4, 5 ) ...
Opening ( 3, 4 ) ...
Progressed ...
Opening ( 3, 5 ) ...
Progressed ...
Opening ( 3, 6 ) ...
Opening ( 2, 5 ) ...
Progressed ...
Opening ( 2, 6 ) ...
Opening ( 1, 5 ) ...
Progressed ...
Opening ( 1, 6 ) ...
Progressed ...
Opening ( 1, 7 ) ...
Progressed ...
Opening ( 2, 7 ) ...
Progressed ...
Opening ( 3, 7 ) ...
Progressed ...
Progressed ...
Opening ( 4, 6 ) ...
Opening ( 4, 7 ) ...
Progressed ...
Opening ( 5, 5 ) ...
Progressed ...
Opening ( 5, 6 ) ...
Progressed ...
Opening ( 5, 7 ) ...
Progressed ...
Progressed ...
Progressed ...
Opening ( 4, 4 ) ...
Progressed ...
Opening ( 5, 4 ) ...
Opening ( 4, 3 ) ...
Progressed ...
Opening ( 5, 3 ) ...
Opening ( 4, 2 ) ...
Progressed ...
Opening ( 5, 2 ) ...
Progressed ...
Opening ( 6, 2 ) ...
Progressed ...
Opening ( 6, 3 ) ...
Opening ( 6, 4 ) ...
Opening ( 6, 5 ) ...
Progressed ...
Opening ( 7, 3 ) ...
Opening ( 7, 2 ) ...
Progressed ...
Opening ( 7, 4 ) ...
Opening ( 7, 5 ) ...
Progressed ...
Opening ( 8, 3 ) ...
Open

2022-01-09 23:55:26.316268: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-09 23:55:26.316448: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-09 23:55:26.348450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-09 23:55:26.434631: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


<class 'numpy.ndarray'>
moves: [[-0.03175908 -0.00131655 -0.00688774  0.0023973  -0.01070425 -0.00117011
   0.01256201 -0.01236676  0.00033783 -0.00330668 -0.00573131 -0.00041618
  -0.01490729 -0.0210507  -0.00324863 -0.00295568  0.00299349 -0.00162585
   0.0036026   0.00557139 -0.00345446  0.0135685   0.00993383 -0.00436229
  -0.00430452  0.00591334  0.00884603 -0.00430308 -0.00136719  0.00862324
   0.01559637  0.01132012  0.00441677 -0.02689847  0.01244349 -0.00212099
   0.01198924  0.00955827 -0.00922583  0.00251326  0.00621188 -0.00431676
   0.01219163  0.00090299 -0.00440909 -0.000742   -0.01405025  0.01378072
   0.01232443  0.0046683   0.01513939 -0.00858422  0.03418166  0.00533576
   0.01738443  0.0189921  -0.02800442 -0.00262195 -0.00401037  0.00722744
  -0.0025406  -0.00624007 -0.00322287  0.02102705  0.02610268 -0.00205782
   0.00459025 -0.00543319  0.02108851  0.00667098  0.0055663   0.01447348
   0.0009596  -0.01237276 -0.00956058 -0.00550001  0.00225086  0.00932209
  -0.01

1/1 [==============================] - 0s 229ms/step - loss: 0.0042
Board:  [[ 0.     0.     0.     0.125 -1.    -1.    -1.    -1.    -1.     0.
   0.     0.     0.125 -1.    -1.    -1.    -1.     0.125  0.     0.
   0.     0.25  -1.    -1.    -1.    -1.     0.125  0.125  0.125  0.
   0.125 -1.    -1.    -1.    -1.    -1.    -1.     0.375  0.125  0.25
  -1.    -1.    -1.     0.25  -1.    -1.    -1.    -1.    -1.    -1.
  -1.    -1.    -1.    -1.    -1.    -1.    -1.     0.5   -1.    -1.
  -1.    -1.    -1.    -1.    -1.    -1.    -1.    -1.    -1.    -1.
  -1.    -1.    -1.    -1.    -1.    -1.    -1.    -1.    -1.    -1.
  -1.   ]]
Unopened Tiles:  [4, 5, 6, 7, 8, 13, 14, 15, 16, 22, 23, 24, 25, 31, 32, 33, 34, 35, 36, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]

Used Random

action:  65
Board:  [[ 0.     0.     0.     0.125 -1.    -1.    -1.    -1.    -1.   ]
 [ 0.     0.  

...

 97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊    | 29/30 [00:06<00:00,  2.86episode/s]

Episode Reward:  0.40000000000000013
Opening ( 1, 3 ) ...
Opening ( 0, 2 ) ...
Opening ( 0, 1 ) ...
Opening ( 0, 0 ) ...
Opening ( 1, 0 ) ...
Opening ( 1, 1 ) ...
Opening ( 1, 2 ) ...
Opening ( 0, 3 ) ...
Opening ( 0, 4 ) ...
Progressed ...
Opening ( 1, 4 ) ...
Progressed ...
Progressed ...
Opening ( 2, 1 ) ...
Progressed ...
Opening ( 2, 2 ) ...
Opening ( 2, 3 ) ...
Opening ( 2, 4 ) ...
Opening ( 1, 5 ) ...
Progressed ...
Opening ( 2, 5 ) ...
Opening ( 1, 6 ) ...
Progressed ...
Opening ( 2, 6 ) ...
Opening ( 1, 7 ) ...
Progressed ...
Opening ( 2, 7 ) ...
Progressed ...
Opening ( 3, 5 ) ...
Progressed ...
Opening ( 3, 6 ) ...
Progressed ...
Opening ( 3, 7 ) ...
Progressed ...
Progressed ...
Opening ( 3, 4 ) ...
Progressed ...
Progressed ...
Opening ( 3, 3 ) ...
Progressed ...
Progressed ...
Opening ( 3, 2 ) ...
Opening ( 3, 1 ) ...
Progressed ...
Opening ( 4, 1 ) ...
Progressed ...
Opening ( 4, 2 ) ...
Opening ( 4, 3 ) ...
Progressed ...
Opening ( 5, 1 ) ...
Opening ( 4, 0 ) ...
Progre

(4, 81)

Future Q Table:  [[-0.18450275 -0.1617541  -0.2537945  -0.04843425  0.01824144 -0.21599856
   0.03885046 -0.29416722 -0.6100486   0.0346456  -0.46489757  0.3478919
  -0.03065661 -0.564278    0.12744333  0.02640806  0.03298055  0.23120552
   0.35751182  0.31703183 -0.26892453  0.10534935  0.60836035  0.06510945
  -0.09795529  0.2363602   0.37749842  0.06455947 -0.10653643 -0.08295359
   0.486768    0.25843745 -0.53462124 -0.20807798  0.05632497 -0.13721055
   0.22980212  0.2958503   0.34010476  0.19501343  0.56101274 -0.05343714
  -0.04866546  0.32004642  0.19907945 -0.00279406 -0.08900501  0.04131752
   0.06107818 -0.01214668  0.15883699 -0.16489035  0.21766083  0.08444247
  -0.10561021  0.5496481  -0.6663989   0.2669489  -0.085743    0.3451984
   0.04558617 -0.02014482 -0.06766049  0.28498504 -0.16437672 -0.00158678
  -0.14076889 -0.3316293  -0.18196246  0.30382147 -0.7146474   0.09041917
   0.11219635 -0.1405798  -0.3372789   0.17610757  0.24824768 -0.39738902
  -0.28463426 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:06<00:00,  4.76episode/s]

Episode Reward:  -1
Number of Wins : 0
